# Pick and Place Project: KUKA KR210 

## Forward Kinematics

In [2]:
from mpmath import *
from sympy import *

# Define DH param symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') #theta
A0, A1, A2, A3, A4, A5, A6 = symbols('A0:7') #alpha
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7') #dist Z(i-1)-Zi along $
d1, d2, d3, d4 ,d5, d6, d7 = symbols('d1:8') #dist X(i-1)-Xi along $
theta2 = symbols('theta2') #theta calculated

#q1, theta2, q3, q4, q5, q6, q7 = 0,0,0,0,0,0,0

DH = {A0:     0, a0:      0, d1: 0.75, 
      A1: -pi/2, a1:   0.35, d2:    0, q2: (theta2-pi/2),
      A2:     0, a2:   1.25, d3:    0, 
      A3: -pi/2, a3: -0.054, d4: 1.50, 
      A4:  pi/2, a4:      0, d5:    0, 
      A5: -pi/2, a5:      0, d6:    0, 
      A6:     0, a6:      0, d7: 0.453, q7:             0
     }

def homogeneous_transform(A,a,d,q):
    out = Matrix([[        cos(q),       -sin(q),       0,         a],
                  [ sin(q)*cos(A), cos(q)*cos(A), -sin(A), -sin(A)*d],
                  [ sin(q)*sin(A), cos(q)*sin(A),  cos(A),  cos(A)*d],
                  [             0,             0,       0,         1]])
    return out

T0_1 = homogeneous_transform(A0,a0,d1,q1)
T0_1 = T0_1.subs(DH)
T1_2 = homogeneous_transform(A1,a1,d2,q2)
T1_2 = T1_2.subs(DH)
T2_3 = homogeneous_transform(A2,a2,d3,q3)
T2_3 = T2_3.subs(DH)
T3_4 = homogeneous_transform(A3,a3,d4,q4)
T3_4 = T3_4.subs(DH)
T4_5 = homogeneous_transform(A4,a4,d5,q5)
T4_5 = T4_5.subs(DH)
T5_6 = homogeneous_transform(A5,a5,d6,q6)
T5_6 = T5_6.subs(DH)
T6_7 = homogeneous_transform(A6,a6,d7,q7)
T6_7 = T6_7.subs(DH)

T0_7 = simplify(T0_1*T1_2*T2_3*T3_4*T4_5*T5_6*T6_7)


In [7]:
T = T3_4*T4_5*T5_6
T[1,2]

cos(q5)

In [ ]:
# individual homogeneous transformation matrices

print('T0_1=')
print(simplify(T0_1))
print('T1_2=')
print(simplify(T1_2))
print('T2_3=')
print(simplify(T2_3))
print('T3_4=')
print(simplify(T3_4))
print('T4_5=')
print(simplify(T4_5))
print('T5_6=')
print(simplify(T5_6))
print('T6_7=')
print(simplify(T6_7))

In [25]:
# Calculate T0_EE using only end effector joint positions (px,py,pz) and orientations (r,p,y)

px = symbols('px')
py = symbols('py')
pz = symbols('pz')
r = symbols('r')
p = symbols('p')
y = symbols('y')

# Defien Euler Angles
R_x = Matrix([[      1,      0,        0],
              [      0, cos(r),  -sin(r)],
              [      0, sin(r),   cos(r)]])
R_y = Matrix([[ cos(p),      0,   sin(p)],
              [      0,      1,        0],
              [-sin(p),      0,   cos(p)]])
R_z = Matrix([[ cos(y),-sin(y),        0],
              [ sin(y), cos(y),        0],
              [      0,      0,        1]])
# Intrinsic rotation
Rxyz = R_x*R_y*R_z 

T0_EE = Rxyz.col_join(Matrix([[0,0,0]])).row_join(Matrix([px,py,pz,1]))
T0_EE

Matrix([
[                        cos(p)*cos(y),                        -sin(y)*cos(p),         sin(p), px],
[ sin(p)*sin(r)*cos(y) + sin(y)*cos(r), -sin(p)*sin(r)*sin(y) + cos(r)*cos(y), -sin(r)*cos(p), py],
[-sin(p)*cos(r)*cos(y) + sin(r)*sin(y),  sin(p)*sin(y)*cos(r) + sin(r)*cos(y),  cos(p)*cos(r), pz],
[                                    0,                                     0,              0,  1]])

In [26]:
### Convert from DH parameter frame to URDF file frame
R_z_corr = Matrix([[   cos(pi),   -sin(pi),           0],
                   [   sin(pi),    cos(pi),           0],
                   [          0,         0,           1]])
R_y_corr = Matrix([[ cos(-pi/2),         0,  sin(-pi/2)],
                   [          0,         1,           0],
                   [-sin(-pi/2),         0,  cos(-pi/2)]])
R_corr = R_z_corr*R_y_corr  ### Intrinsic rotation
Rrpy = simplify(T0_EE[0:3,0:3]*R_corr.inv())
Rrpy

###########################output############################
# Rrpy = 
# Matrix([
# [        sin(p),                         sin(y)*cos(p),                         cos(p)*cos(y)],
# [-sin(r)*cos(p),  sin(p)*sin(r)*sin(y) - cos(r)*cos(y),  sin(p)*sin(r)*cos(y) + sin(y)*cos(r)],
# [ cos(p)*cos(r), -sin(p)*sin(y)*cos(r) - sin(r)*cos(y), -sin(p)*cos(r)*cos(y) + sin(r)*sin(y)]])

Matrix([
[        sin(p),                         sin(y)*cos(p),                         cos(p)*cos(y)],
[-sin(r)*cos(p),  sin(p)*sin(r)*sin(y) - cos(r)*cos(y),  sin(p)*sin(r)*cos(y) + sin(y)*cos(r)],
[ cos(p)*cos(r), -sin(p)*sin(y)*cos(r) - sin(r)*cos(y), -sin(p)*cos(r)*cos(y) + sin(r)*sin(y)]])

## Inverse Kinematics

In [27]:
# Calculate Rrpy Rotation Matrix given roll pitch and yaw
## Defien Euler Angles
R_x = Matrix([[      1,      0,        0],
              [      0, cos(r),  -sin(r)],
              [      0, sin(r),   cos(r)]])
R_y = Matrix([[ cos(p),      0,   sin(p)],
              [      0,      1,        0],
              [-sin(p),      0,   cos(p)]])
R_z = Matrix([[ cos(y),-sin(y),        0],
              [ sin(y), cos(y),        0],
              [      0,      0,        1]])
R0_EE = simplify(R_x*R_y*R_z)  ### Intrinsic rotation

### Convert from DH parameter frame to URDF file frame
R_z_corr = Matrix([[   cos(pi),   -sin(pi),           0],
                   [   sin(pi),    cos(pi),           0],
                   [          0,         0,           1]])
R_y_corr = Matrix([[ cos(-pi/2),         0,  sin(-pi/2)],
                   [          0,         1,           0],
                   [-sin(-pi/2),         0,  cos(-pi/2)]])
R_corr = R_z_corr*R_y_corr  ### Intrinsic rotation
Rrpy = simplify(R0_EE*R_corr.inv())
Rrpy

###########################output############################
# Matrix([
# [        sin(p),                         sin(y)*cos(p),                         cos(p)*cos(y)],
# [-sin(r)*cos(p),  sin(p)*sin(r)*sin(y) - cos(r)*cos(y),  sin(p)*sin(r)*cos(y) + sin(y)*cos(r)],
# [ cos(p)*cos(r), -sin(p)*sin(y)*cos(r) - sin(r)*cos(y), -sin(p)*cos(r)*cos(y) + sin(r)*sin(y)]])

Matrix([
[        sin(p),                         sin(y)*cos(p),                         cos(p)*cos(y)],
[-sin(r)*cos(p),  sin(p)*sin(r)*sin(y) - cos(r)*cos(y),  sin(p)*sin(r)*cos(y) + sin(y)*cos(r)],
[ cos(p)*cos(r), -sin(p)*sin(y)*cos(r) - sin(r)*cos(y), -sin(p)*cos(r)*cos(y) + sin(r)*sin(y)]])

## Calculate q1, q2, and q3

In [ ]:
# import rospy
# import tf
# from kuka_arm.srv import *
# from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
# from geometry_msgs.msg import Pose
from mpmath import *
from sympy import *

# Define DH param symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') #theta ###real=True
A0, A1, A2, A3, A4, A5, A6 = symbols('A0:7') #alpha
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7') #dist Z(i-1)-Zi along X(i-1)
d1, d2, d3, d4 ,d5, d6, d7 = symbols('d1:8') #dist X(i-1)-Xi along Zi
theta2 = symbols('theta2') #theta2 calculated

# DH params for Kuka KR210
DH = {A0:     0, a0:      0, d1:  0.75, 
      A1: -pi/2, a1:   0.35, d2:     0, q2: (theta2-pi/2),
      A2:     0, a2:   1.25, d3:     0, 
      A3: -pi/2, a3: -0.054, d4:  1.50, 
      A4:  pi/2, a4:      0, d5:     0, 
      A5: -pi/2, a5:      0, d6:     0, 
      A6:     0, a6:      0, d7: 0.303, q7:             0
     }

# Extract end-effector position and orientation from request
# px,py,pz = end-effector position
# roll, pitch, yaw = end-effector orientation
px,py,pz = 1.5, -0.2, 1.9
r,p,y = .5,.5,.5
# r = symbols('r')
# p = symbols('p')
# y = symbols('y')

########################### Calculate joint angles using Geometric IK method ###########################

# Rotation matrix from base frame to WC. obtained from r,p,y
Rrpy = Matrix([
              [        sin(p),                         sin(y)*cos(p),                         cos(p)*cos(y)],
              [-sin(r)*cos(p),  sin(p)*sin(r)*sin(y) - cos(r)*cos(y),  sin(p)*sin(r)*cos(y) + sin(y)*cos(r)],
              [ cos(p)*cos(r), -sin(p)*sin(y)*cos(r) - sin(r)*cos(y), -sin(p)*cos(r)*cos(y) + sin(r)*sin(y)]])

# Calculate wrist center position
l = 0
wx = float(px - (d7.subs(DH)+l)*Rrpy[0,2]
wy = float(py - (d7.subs(DH)+l)*Rrpy[1,2]
wz = float(pz - (d7.subs(DH)+l)*Rrpy[2,2]
           
## calculate q1
q1 = float(atan2(wy,wx))

## Calculate q2 and q3 using the Laws of Cosine
### calculate distance between WC and joint_2 projected on to xy plane
xybase = sqrt((wx)**2+(wy)**2)-a1.subs(DH)
### distance from joint_2 to wrist center
d2_WC = sqrt((wz-d1.subs(DH))**2+(xybase)**2)
### check if d2_WC is within joint_3 angle limitations -90 65 (mechanical limitation = -210 65)
if (d2_WC < 0.643863):
    print('d2_WC < invalid d2_WC')
if (d2_WC > 2.750097):
    print('d2_WC > invalid d2_WC')
    print(px,py,pz,r,p,y)
    d2_WC = 2.75
### angle between d2_wc and x-y plane of base_link
AWC_2_xybase = atan2((wz-d1.subs(DH)),(xybase))
### distance from joint_2 to joint_3
d2_3 = 1.25 # = a2.subs(DH)
### distance from joint_3 to wrist center
d3_WC = 1.50097 # = sqrt(d4.subs(DH)**2+a3.subs(DH)**2)
### angle between d2_3 and d3_WC
A2_3_WC = acos((3.81541-d2_WC**2)/(3.75243)) # = acos((d2_3**2+d3_WC**2-d2_WC**2)/(2*d2_3*d3_WC))
### angle between d3_WC and d4
delta = atan2(a3.subs(DH),d4.subs(DH))
### angle between d2_3 and d2_WC using The Laws of Sines
if (d2_WC > 0.643863)|(d2_WC <= 1.95331):
    A3_2_WC = asin(d3_WC/d2_WC*sin(A2_3_WC))
elif (d2_WC > 1.95331)|(d2_WC < 2.75097):
    A3_2_WC = pi - asin(d3_WC/d2_WC*sin(A2_3_WC))
### q2 and q3
Q2 = float(pi/2.-A3_2_WC-AWC_2_xybase)
q3 = float(pi/2.-A2_3_WC-delta)           

## Calculate q4, q5, and q6

In [ ]:
# Calculate q4 q5 and q6

## Calculate R0_3 Rotation matrix from T0_3 Homogeneous Transformation matrix
def homogeneous_transform(A,a,d,q):
    out = Matrix([[        cos(q),       -sin(q),       0,         a],
                 [ sin(q)*cos(A), cos(q)*cos(A), -sin(A), -sin(A)*d],
                 [ sin(q)*sin(A), cos(q)*sin(A),  cos(A),  cos(A)*d],
                 [             0,             0,       0,         1]])
    return out

T0_1 = homogeneous_transform(A0,a0,d1,q1)
T0_1 = T0_1.subs(DH)
T0_1 = T0_1.subs(q1,q1)
T1_2 = homogeneous_transform(A1,a1,d2,q2)
T1_2 = T1_2.subs(DH)
T1_2 = T1_2.subs(theta2,Q2)
T2_3 = homogeneous_transform(A2,a2,d3,q3)
T2_3 = T2_3.subs(DH)
T2_3 = T2_3.subs(q3,q3)
T0_3 = T0_1*T1_2*T2_3
R0_3 = T0_3[0:3,0:3] # convert from Homogeneous transform matrix to rotation matrix

### Calculate R3_6 (Rrpy = R0_6)
R3_6 = simplify(R0_3.inv()*Rrpy)
# R3_6 = 
# Matrix([
# [(-(sp*sr*cy+sy*cr)*s(q1)*s(q3+t2)**2-(sp*cr*cy-sr*sy)*(-c(q3+t2)**2+1)*c(q1)+s(q3+t2)*cp*cy*c(q3+theta2))/s(q3+t2), (-(-sp*sr*sy+cr*cy)*s(q1)*s(q3+t2)**2+(sp*sy*cr+sr*cy)*(-c(q3+theta2)**2+1)*c(q1)-sy*s(q3+t2)*cp*c(q3+t2))/s(q3+t2), (2*c(-p+2*q3+2*t2)+2*c(p-q1+r)+2*c(p+q1-r)-2*c(p+2*q3+2*t2)-c(-p-q1+2*q3+r+2*t2)-c(-p+q1+2*q3-r+2*t2)-c(p-q1+2*q3+r+2*t2)-c(p+q1+2*q3-r+2*t2))/(8*s(q3+t2))],
# [ -sp*cy*c(q1-r)*c(q3+t2)-sy*s(q1-r)*c(q3+t2)-s(q3+t2)*cp*cy, sp*sy*c(q1-r)*c(q3+t2)+sy*s(q3+t2)*cp-s(q1-r)*cy*c(q3+t2), -sp*s(q3+t2)+cp*c(q1-r)*c(q3+t2)],
# [  sp*s(q1-r)*cy-sy*c(q1-r),     -sp*sy*s(q1-r)-cy*c(q1-r),      -s(q1-r)*cp]])

### equate above R3_6 with R3_6 obtained from T3_6 with DH parameters
### Obtain system of equation by letting T3_6[0:3,0:3]-R3_6 = 0 and solve for q4,q5,q6
### R3_6 obtained from T3_6 with DH parameters
# R3_6 = 
# Matrix([
# [-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -sin(q5)*cos(q4)],
# [                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5)],
# [-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5)]])

# R3_6[1,2] = -sp*s(q3+t2)+cp*c(q1-r)*c(q3+t2)
# R3_6[2,2] = -s(q1-r)*cp
# R3_6[0,2] = (2*c(-p+2*q3+2*t2)+2*c(p-q1+r)+2*c(p+q1-r)-2*c(p+2*q3+2*t2)-c(-p-q1+2*q3+r+2*t2)-c(-p+q1+2*q3-r+2*t2)-c(p-q1+2*q3+r+2*t2)-c(p+q1+2*q3-r+2*t2))/(8*s(q3+t2))
# R3_6[1,1] = sp*sy*c(q1-r)*c(q3+t2)+sy*s(q3+t2)*cp-s(q1-r)*cy*c(q3+t2)
# R3_6[1,0] = -sp*cy*c(q1-r)*c(q3+t2)-sy*s(q1-r)*c(q3+t2)-s(q3+t2)*cp*cy

### Solve for q5 using the equation cos(q5)-R3_6[1,2] = 0
cos_q5 = R3_6[1,2]
#Q5 = float(acos(R3_6[1,2]))
Q5 = atan2(sqrt(1 - cos_q5**2), cos_q5)
### Solve for q4 using the equation sin(q4)*sin(q5)-R3_6[2,2]=0 & -sin(q5)*cos(q4)-R3_6[0,2]=0
Q4 = float(atan2(R3_6[2,2],(-R3_6[0,2])))
### Solve for q6 using the equation sin(q5)*cos(q6)-R3_6[1,0]=0 & -sin(q5)*sin(q6)-R3_6[1,1]=0
Q6 = float(atan2((-R3_6[1,1]),R3_6[1,0]))

### Joint Mechanical Limits

In [ ]:
### joint limit

joint1 = -185 185
joint2 = -45 85
joint3 = -210 155-90=65
joint4 = -350 350
joint5 = -125 125
joint6 = -350 350

### R3_6 from FK (q4, q5, q6 as variables)

In [ ]:
### calculate R3_6 from FK

T3_4 = homogeneous_transform(A3,a3,d4,q4)
T3_4 = T3_4.subs(DH)
T4_5 = homogeneous_transform(A4,a4,d5,q5)
T4_5 = T4_5.subs(DH)
T5_6 = homogeneous_transform(A5,a5,d6,q6)
T5_6 = T5_6.subs(DH)
T3_6 = simplify(T3_4*T4_5*T5_6)
R3_6 = T3_6[0:3,0:3]
R3_6

### R3_6 from IK (r, p, y, q1, q2, q3 as variables)

In [ ]:
### calculate R3_6 from IK

T0_3 = simplify(T0_1*T1_2*T2_3)
R0_3 = T0_3[0:3,0:3]  ### convert from Homogeneous transform matrix to rotation matrix

r = symbols('r')
p = symbols('p')
y = symbols('y')

# Calculate joint angles using Geometric IK method

## Calculate Rrpy Rotation Matrix given roll pitch and yaw
### Defien Euler Angles
R_x = Matrix([[      1,      0,        0],
              [      0, cos(r),  -sin(r)],
              [      0, sin(r),   cos(r)]])
R_y = Matrix([[ cos(p),      0,   sin(p)],
              [      0,      1,        0],
              [-sin(p),      0,   cos(p)]])
R_z = Matrix([[ cos(y),-sin(y),        0],
              [ sin(y), cos(y),        0],
              [      0,      0,        1]])
Rxyz = simplify(R_x*R_y*R_z)  ### Intrinsic rotation

### Convert from DH parameter frame to URDF file frame
R_z_corr = Matrix([[   cos(pi),   -sin(pi),           0],
                   [   sin(pi),    cos(pi),           0],
                   [          0,         0,           1]])
R_y_corr = Matrix([[ cos(-pi/2),         0,  sin(-pi/2)],
                   [          0,         1,           0],
                   [-sin(-pi/2),         0,  cos(-pi/2)]])
R_corr = R_z_corr*R_y_corr  ### Intrinsic rotation
Rrpy = simplify(R_corr.inv()*Rxyz)

### Calculate R3_6 (Rrpy = R0_6)
R3_6 = simplify(R0_3.inv()*Rrpy)
R3_6

### Test

In [1]:
from mpmath import *
from sympy import *
import numpy as np

In [ ]:
#!/usr/bin/env python

# Copyright (C) 2017 Electric Movement Inc.
#
# This file is part of Robotic Arm: Pick and Place project for Udacity
# Robotics nano-degree program
#
# All Rights Reserved.

# Author: Harsh Pandya

# import modules
import rospy
import tf
from kuka_arm.srv import *
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from geometry_msgs.msg import Pose
from mpmath import *
from sympy import *
from sympy.physics.vector import dot, cross
import numpy as np
from numpy.linalg import norm

def handle_calculate_IK(req):
    rospy.loginfo("Received %s eef-poses from the plan" % len(req.poses))
    if len(req.poses) < 1:
        print "No valid poses received"
        return -1
    else:
		# Initialize service response
		joint_trajectory_list = []
		# Initialize 
		prevq4,prevq5,prevq6 = 0,0,0

		for x in xrange(0, len(req.poses)):
			# IK code starts here
			joint_trajectory_point = JointTrajectoryPoint()

			# Initialize
			theta1,theta2,theta3,theta4,theta5,theta6 = 0,0,0,0,0,0

			# Define DH param 
			A0, A1, A2, A3, A4, A5, A6 = 0,-np.pi/2,0,-np.pi/2,np.pi/2,-np.pi/2,0
			a0, a1, a2, a3, a4, a5, a6 = 0,.35,1.25,-0.054,0,0,0
			d1, d2, d3, d4 ,d5, d6, d7 = .75,0,0,1.5,0,0,.453

			# Extract end-effector position and orientation from request
			# px,py,pz = end-effector position
			# roll, pitch, yaw = end-effector orientation
			px = req.poses[x].position.x
			py = req.poses[x].position.y
			pz = req.poses[x].position.z

			r, p, y = tf.transformations.euler_from_quaternion(
				[req.poses[x].orientation.x, req.poses[x].orientation.y,
					req.poses[x].orientation.z, req.poses[x].orientation.w])

			# Rrpy = R0_6 = R0_EE*R6_EE.inv()
			Rrpy = Matrix([
			  			  [        sin(p),                         sin(y)*cos(p),                         cos(p)*cos(y)],
			  			  [-sin(r)*cos(p),  sin(p)*sin(r)*sin(y) - cos(r)*cos(y),  sin(p)*sin(r)*cos(y) + sin(y)*cos(r)],
			  			  [ cos(p)*cos(r), -sin(p)*sin(y)*cos(r) - sin(r)*cos(y), -sin(p)*cos(r)*cos(y) + sin(r)*sin(y)]])

			## Calculate wrist center position
			l = 0  
			wx = float(px - (d7+l)*Rrpy[0,2])
			wy = float(py - (d7+l)*Rrpy[1,2])
			wz = float(pz - (d7+l)*Rrpy[2,2])

			### check wrist position
			#w_pos_error = sqrt((px-wx)**2+(py-wy)**2+(pz-wz)**2)
			#print('wrist position error')
			#print(float(w_pos_error))

			### Calculate first three joint angles, q1,q2,and q3 using Geometric IK method
			###  q1
			theta1 = float(atan2(wy,wx))

			### Calculate q2 and q3 using the Laws of Cosine
			### calculate distance between WC and joint_2 projected on to xy plane
			xybase = sqrt((wx)**2+(wy)**2)-a1
			### distance from joint_2 to wrist center
			d2_WC = sqrt((wz-d1)**2+(xybase)**2)
			### check if d2_WC is within joint_3 angle limitations -90 65 (mechanical limitation = -210 65)
			if (d2_WC < 0.643863):
				print('d2_WC < invalid d2_WC')
				d2_WC = 0.644
			if (d2_WC > 2.750097):
				print('d2_WC > invalid d2_WC')
				d2_WC = 2.75
			### angle between d2_wc and x-y plane of base_link
			AWC_2_xybase = atan2((wz-d1),(xybase))
			### distance from joint_2 to joint_3
			d2_3 = a2
			### distance from joint_3 to wrist center
			d3_WC = sqrt(d4**2+a3**2)
			### angle between d2_3 and d3_WC
			A2_3_WC = acos((d2_3**2+d3_WC**2-d2_WC**2)/(2*d2_3*d3_WC))
			### angle between d3_WC and d4
			delta = atan2(a3,d4)
			### angle between d2_3 and d2_WC using The Laws of Sines
			if (d2_WC > 0.643863)|(d2_WC <= 1.95331):
				A3_2_WC = asin(d3_WC/d2_WC*sin(A2_3_WC))
			elif (d2_WC > 1.95331)|(d2_WC < 2.75097):
				A3_2_WC = np.pi - asin(d3_WC/d2_WC*sin(A2_3_WC))
			### q2 and q3
			theta2 = float(np.pi/2.-A3_2_WC-AWC_2_xybase)
			theta3 = float(np.pi/2.-A2_3_WC-delta)

			## Calculate q4 q5 and q6
			### Calculate R0_3 Rotation matrix from T0_3 Homogeneous Transformation matrix
			def homogeneous_transform(A,a,d,q):
				out = Matrix([[        cos(q),       -sin(q),       0,         a],
							  [ sin(q)*cos(A), cos(q)*cos(A), -sin(A), -sin(A)*d],
							  [ sin(q)*sin(A), cos(q)*sin(A),  cos(A),  cos(A)*d],
							  [             0,             0,       0,         1]])
				return out

			T0_1 = homogeneous_transform(A0,a0,d1,theta1)
			T1_2 = homogeneous_transform(A1,a1,d2,theta2-np.pi/2)
			T2_3 = homogeneous_transform(A2,a2,d3,theta3)
			T0_3 = T0_1*T1_2*T2_3
			
			### tried but didn't use for final submission
			### Calculate R3_6		
			R0_3 = T0_3[0:3,0:3] # from Homogeneous transform matrix to rotation matrix
			R3_6 = R0_3.inv()*Rrpy
			## equate above R3_6 with R3_6 obtained from T3_6 with DH parameters
			## Obtain system of equation by letting T3_6[0:3,0:3]-R3_6 = 0 and solve for q4,q5,q6
			## R3_6 obtained from T3_6 with DH parameters
			R3_6 = 
			Matrix([
			[-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -sin(q5)*cos(q4)],
			[                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5)],
			[-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5)]])
			## Solve for q5: sin(q5)*cos(q6)-R3_6[1,0]=0 & -sin(q5)*sin(q6)-R3_6[1,1]=0 & cos(q5)-R3_6[1,2]=0
			## Solve for q4: sin(q4)*sin(q5)-R3_6[2,2]=0 & -sin(q5)*cos(q4)-R3_6[0,2]=0			
			## Solve for q6 using the equation sin(q5)*cos(q6)-R3_6[1,0]=0 & -sin(q5)*sin(q6)-R3_6[1,1]=0			
			
						
			### tried but didn't use for final submission
			#q4, q5, q6 = tf.transformations.euler_from_matrix(np.array(R3_6).astype(np.float64),axes='ryzy')
			
			
			### Solve for q5
			T3_4 = homogeneous_transform(A3,a3,d4,0)
			T0_4 = T0_3*T3_4
			nz4 = Matrix([T0_4[0,2], T0_4[1,2], T0_4[2,2]]).transpose()
			nz6 = Matrix([Rrpy[0,2], Rrpy[1,2], Rrpy[2,2]])
			xx = (nz4*nz6)/norm(nz4)
			x = (xx/norm(nz6))[0,0]			
			Q5 = []
			Q5.append(np.clip(2*atan2(sqrt(1-x**2),(1+x)),-np.pi*124/180,np.pi*124/180))
			Q5.append(np.clip(2*atan2(-sqrt(1-x**2),(1+x)),-np.pi*124/180,np.pi*124/180))
			Q5.append(-Q5[0])
			Q5.append(-Q5[1])
			
			### Solve for q4
			nx4 = Matrix([T0_4[0,0], T0_4[1,0], T0_4[2,0]]).transpose()
			nx6 = Matrix([Rrpy[0,0], Rrpy[1,0], 0])
			xx = (nx4*nx6)/norm(nx4)
			x = (xx/norm(nx6))[0,0]
			Q4 = []
			Q4.append(2*atan2(sqrt(1-x**2),(1+x)))
			Q4.append(2*atan2(-sqrt(1-x**2),(1+x)))
			Q4.append(float(np.pi-Q4[0]))
			Q4.append(float(np.pi-Q4[1]))	
					
			### Solve for q6
			T3_4 = homogeneous_transform(A3,a3,d4,theta4)
			T4_5 = homogeneous_transform(A4,a4,d5,theta5)
			T5_6 = homogeneous_transform(A5,a5,d6,0)
			T0_6 = T0_3*T3_4*T4_5*T5_6	
			nx06 = Matrix([T0_6[0,0], T0_6[1,0], T0_6[2,0]]).transpose()
			nx6 = Matrix([Rrpy[0,0], Rrpy[1,0], Rrpy[2,0]])	
			Q6 = []
			x = (nx06*nx6)[0,0]
			Q6.append(2*atan2(sqrt(1-x**2),(1+x)))
			Q6.append(2*atan2(-sqrt(1-x**2),(1+x)))
			Q6.append(float(np.pi-Q6[0]))
			Q6.append(float(np.pi-Q6[1]))		
											
            
            ### Find the best combination of q4, q5, and q6 by Calculation position error using FK
			pos_error = 100 # can be any large number
			compare = 100 # can be any large number
			for temp5 in Q5:
				for temp4 in Q4:
					for temp6 in Q6:
						###### check and compare with FK #######
						T3_4 = homogeneous_transform(A3,a3,d4,temp4)
						T4_5 = homogeneous_transform(A4,a4,d5,temp5)
						T5_6 = homogeneous_transform(A5,a5,d6,temp6)
						T6_7 = homogeneous_transform(A6,a6,d7,0)
						T0_6 = T0_3*T3_4*T4_5*T5_6
						T0_7 = T0_6*T6_7
						### use Pythagorean theorem to find position error	    
						pos_error_new = sqrt((T0_7[0,3]-px)**2 + (T0_7[1,3]-py)**2 + (T0_7[2,3]-pz)**2)
						print(pos_error)
						if pos_error_new <= pos_error:
							pos_error = pos_error_new					
							compare_new = abs(temp4-prevq4) + abs(temp5-prevq5) #+ abs(temp6-prevq6)
							if compare_new<compare:
								theta5 = temp5
								theta4 = temp4
								theta6 = temp6
                                
			### add correction to q4 angle based on previous joint angle state
			while abs(float(theta4-prevq4)) >= np.pi/2:
				if theta4-prevq4 > 0:
					theta4 = theta4-np.pi
					theta5 = -theta5
				else:
					theta4 = theta4+np.pi
					theta5 = -theta5	
			### add correction to q6 angle based on previous joint angle state
			while abs(float(theta6-0)) >= np.pi/2:
				if theta6-prevq6 > 0:
					theta6 = theta6-np.pi
				else:
					theta6 = theta6+np.pi


			###### check and compare with FK #######
			T3_4 = homogeneous_transform(A3,a3,d4,theta4)
			T0_4 = T0_3*T3_4
			### use Pythagorean theorem to find wrist position error
			#wrist_pos_error = sqrt((T0_4[0,3]-px)**2 + (T0_4[1,3]-py)**2 + (T0_4[2,3]-pz)**2)
			T4_5 = homogeneous_transform(A4,a4,d5,theta5)
			T5_6 = homogeneous_transform(A5,a5,d6,theta6)
			T6_7 = homogeneous_transform(A6,a6,d7,0)
			T0_7 = T0_4*T4_5*T5_6*T6_7
			### use Pythagorean theorem to find position error	    
			pos_error = sqrt((T0_7[0,3]-px)**2 + (T0_7[1,3]-py)**2 + (T0_7[2,3]-pz)**2)
			print('finalerror')
			print(pos_error)

			# Populate response for the IK request
			# In the next line replace theta1,theta2...,theta6 by your joint angle variables
			joint_trajectory_point.positions = [theta1, theta2, theta3, theta4, theta5, theta6]
			joint_trajectory_list.append(joint_trajectory_point)

			prevq4,prevq5,prevq6 = theta4,theta5,theta6

		rospy.loginfo("length of Joint Trajectory List: %s" % len(joint_trajectory_list))
		return CalculateIKResponse(joint_trajectory_list)


def IK_server():
    # initialize node and declare calculate_ik service
    rospy.init_node('IK_server')
    s = rospy.Service('calculate_ik', CalculateIK, handle_calculate_IK)
    print "Ready to receive an IK request"
    rospy.spin()

if __name__ == "__main__":
    IK_server()

In [ ]:
[INFO] [1499472644.016838, 1206.271000]: Received 37 eef-poses from the plan
q5 okay
(1.14766430375217e-07, -0.08824712435140109, 0.15697355267209107, -3.141592653589791, 0.06872614831583246, -3.1415911263143745)
(2.042999799843439, 2.3446778281719667e-07, 1.9459991997808317, -1.4394862279942455e-07, 2.8000484153523107e-07, 1.1476643283294864e-07)
q5 okay
(-0.013668207548039024, -0.08413522850945027, 0.19118322767113233, 3.1355223273135207, 0.09273730904411313, -3.132668546462224)
(2.0449169104735936, -0.027781746409740675, 1.8846665681788433, 0.003075542481362998, 0.01427324602621294, -0.013148630419328655)
q5 okay
(-0.027411702385550348, -0.08007972274627123, 0.22542056402705662, 3.140889286410992, 0.11663999593316719, 2.9335885652780602)
(2.0441360887703115, -0.055366926068236996, 1.8234778544804127, 0.006228331121445953, 0.02857416298112749, -0.02533424921863825)
q5 okay
(-0.04123073587510087, -0.07608299266765556, 0.2596845131489674, 0.0012500042832543966, 0.1404423121733599, 2.8953106271605575)
(2.0406741804053774, -0.08265620909335837, 1.7625175044752313, 0.009535296384704005, 0.04293053619216286, -0.03655781770532091)
q5 off
(-0.05512712884593368, -0.07214755983766791, 0.2939730428084955, 0.0014630395731620699, 0.16415202894025538, 1.903285809060212)
(2.0345532711581678, -0.10955169147639393, 1.701869758343593, 0.013073458476924718, 0.05736898366576528, -0.046821067772001276)
q5 okay
(-0.0625762398577878, -0.06992773987305195, 0.31197146606052867, 3.076091986908453, 0.17680500062824855, -3.066238032270491)
(2.030443044490323, -0.1237285063785036, 1.6699747694043867, 0.014989716809367849, 0.06492119049608003, -0.051907145281677496)
q5 okay
(-0.07004857237874856, -0.06772535725604006, 0.329974960532931, 3.0651583546218273, 0.18943510567848812, -3.054672064373134)
(2.025599797394347, -0.13775539359259958, 1.6382019731420492, 0.017004510981111002, 0.07250769452698802, -0.056720682044586856)
q5 okay
(-0.07754490500670033, -0.06554085132064741, 0.34798300807998156, 3.0539304046113434, 0.20204299794264938, -3.042893842708338)
(2.0200285496257333, -0.15161827332894115, 1.6065637089086404, 0.019129361281291685, 0.08013199816003294, -0.06126214884818135)
q5 okay
(-0.08506614396480278, -0.06337468062092427, 0.3659950602810927, 3.0424346451709634, 0.21462938548000815, -3.030927084306674)
(2.013734719372037, -0.16530321682035756, 1.575072290535803, 0.021375776554945377, 0.08779742804627656, -0.0655320445582792)
q5 okay
(-0.09716108273854726, -0.05887212678119263, 0.3926028589584201, 3.127153998340493, 0.23501396931084578, 2.860219374661816)
(2.0040548082454377, -0.18702031774386801, 1.5271760995221293, 0.024564420110803654, 0.09875555829005962, -0.07181329445755316)
q5 okay
(-0.10932184634797477, -0.054404887663100636, 0.4192098492510408, 3.1028883505415656, 0.2553559111767315, 2.863596979534767)
(1.9926898395591197, -0.20824772091546537, 1.4796839402729431, 0.028125219873695987, 0.1098397338683375, -0.07740206707226918)
q5 off
(-0.12537096560742067, -0.045532945844517884, 0.4478150952006508, 3.1300137195056545, 0.28279156855513177, -3.12016069375735)
(1.9809435924161207, -0.23606751389557878, 1.4239970515018916, 0.031442898789340444, 0.12063908746285602, -0.08371995047232907)
q5 okay
(-0.14153291616619124, -0.036676475418168035, 0.4763861160587726, 2.956815537659598, 0.31018197309464945, -2.949110970233603)
(1.9668105374683766, -0.2630784597155191, 1.3689438353567973, 0.03550089145730837, 0.13169843104336745, -0.0888638346572471)
q5 okay
(-0.16555361233806418, -0.01888486276130468, 0.5088359117660893, 2.9219817852174645, 0.3517723249325146, -2.919525846336613)
(1.951025647468774, -0.3029236641561882, 1.2976006224792258, 0.039828613284931036, 0.14251479755952917, -0.09439450819710861)
q5 okay
(-0.18982563119773005, -0.0010326384299330026, 0.5411396024878984, 2.885717860200445, 0.3933191889594306, -2.888744226548989)
(1.9310788253341806, -0.34110730167588305, 1.2274088561702747, 0.046001749888879416, 0.1539852210585235, -0.09746801694690907)
q5 off
(-0.2213165024946593, 0.03362343409406338, 0.5594979207742511, 0.14838223704875358, 0.4498123157385483, 2.9471260120714433)
(1.9232727637251579, -0.39308164498614023, 1.1575481760771094, 0.04877572793464082, 0.15549595956244616, -0.09811934328037637)
q5 okay
(-0.25315257006498776, 0.06857035328395972, 0.5774615422652439, 2.799314221408635, 0.5062452112286314, -2.8266544747050846)
(1.910641304635258, -0.4433520771782438, 1.0881208022623565, 0.05457783101894435, 0.158217934085848, -0.09505285712925146)
q5 off
(-0.29131283878931824, 0.1283174773445544, 0.563397183110598, 2.9092670132238716, 0.5778522107114512, 3.017352570475888)
(1.926845595190422, -0.5122961816799583, 1.030595255285423, 0.051848579979380624, 0.14165482564540996, -0.08720467412614445)
q5 off
(-0.3297586410550318, 0.18839843103486836, 0.5485986200563336, 2.862412510079622, 0.6489134913003153, 3.1278413594528214)
(1.937577151239868, -0.5811774490500489, 0.9703489425242577, 0.052468462825959954, 0.12703596026597916, -0.07482296926648627)
q5 okay
(-0.36170431170333683, 0.24464103288087707, 0.5232054025729963, 2.6647068036978867, 0.7086914900947278, -2.7593458529694783)
(1.955527763679237, -0.6430915122754133, 0.9260031324160454, 0.05002247812778107, 0.10884199861780981, -0.06152326168688724)
q5 okay
(-0.3937435847796491, 0.30098099700344166, 0.4972490267669257, 2.623804289252601, 0.7678469402052713, -2.7416978702048675)
(1.9693960652708054, -0.7053607044710957, 0.8788537939773599, 0.04955730932694242, 0.09200897579482507, -0.04558908266854154)
q5 off
(-0.40631735233589616, 0.324767182671685, 0.4835188225180263, 1.409176450701497, 0.791198731111702, 3.132167419512391)
(1.9772187098766174, -0.7313846144545012, 0.8614243606775659, 0.0482904100985763, 0.08378433921144458, -0.03868214480779033)
q5 off
(-0.4188979830770591, 0.34855255650362565, 0.46969601476270856, 2.879143239009466, 0.8144259417231479, -3.116797175361212)
(1.9843421022780932, -0.7574945428697671, 0.843467143141998, 0.04728053088512536, 0.07577640362040482, -0.031408914297572574)
q5 okay
(-0.42172336308723013, 0.3560157176879964, 0.462241073976369, 2.5887476427455023, 0.8200218962196058, -2.730567350018513)
(1.9903104930018731, -0.7654299818527796, 0.841568665715181, 0.04493537591291676, 0.07150914530313777, -0.029729776512532443)
q5 okay
(-0.4245516640880286, 0.3634734107163145, 0.45477979537060437, 2.58558174567425, 0.8256058498056777, -2.7308349684686006)
(1.9962198373130393, -0.7733886253495522, 0.8396129029969897, 0.04258315779692219, 0.06726162455894431, -0.02802744028014898)
q5 off
(-0.4254832561719064, 0.3660848539821691, 0.45199898989680937, 2.4319383706423796, 0.8274714323158062, -3.1390856896027763)
(1.998483074781982, -0.7761676022932856, 0.8391200046704782, 0.04164556897501113, 0.06568086786162573, -0.027463068456771284)
q5 okay
(-0.4264152864719649, 0.3686955604613134, 0.44921743067355596, 2.583546334210352, 0.8293356730362463, -2.731234373460704)
(2.000738991438641, -0.778949416357229, 0.8386202225586445, 0.040706692253990304, 0.06410249776678718, -0.026895976642604905)
q5 okay
(-0.42734775230899547, 0.3713055348767971, 0.446435113302673, 2.5825189801635395, 0.8311985666939842, -2.7314297703345187)
(2.002987570406492, -0.781734049092548, 0.8381135619045562, 0.03976652465987008, 0.06252652065177859, -0.02632616361470793)
q5 off
(-0.428280651015715, 0.37391478189953925, 0.4436520333953982, 2.877592583211365, 0.8330601080040336, 2.1863145189166833)
(2.0052287948702143, -0.7845214820081152, 0.8376000279953286, 0.03882506324812934, 0.06095294290086593, -0.02575362815836989)
q5 off
(-0.4341212769287917, 0.39024647383618044, 0.4262061061088394, 1.4826224222719073, 0.8446711067593452, -1.8464837314655147)
(2.01910486502016, -0.8020374452416087, 0.8342161643086833, 0.03291584872784404, 0.051149600861132, -0.022161705796802184)
q5 off
(-0.4399781290325749, 0.4065510172367888, 0.40872925118195164, 2.9364205872558355, 0.8562286511448535, -3.140700141470006)
(2.032688045704063, -0.819657900334488, 0.8305642421870584, 0.02695509018359176, 0.041440732927659506, -0.018463788630492615)
q5 off
(-0.4458505867225708, 0.4228294341470729, 0.39122041755834164, 2.7371494523460074, 0.8677314339336919, -3.141257275192687)
(2.0459743690111605, -0.8373781906917444, 0.8266456864820574, 0.02094213943366931, 0.0318278979847251, -0.014659584807041456)
q5 okay
(-0.45173804307411447, 0.43908267650239036, 0.37367857108073427, 2.5539225882643075, 0.8791781278874241, -2.7357942787439673)
(2.05895996450521, -0.8551935987989999, 0.8224619891574797, 0.014876394911238537, 0.02231266437584515, -0.010748819802669982)
q5 off
(-0.4554148157929313, 0.44920838096972676, 0.3627215159322475, 0.4100255838251444, 0.8862851393386991, 1.9041513902258551)
(2.0669064759847555, -0.8663497839670186, 0.8197167839125352, 0.011069796662419364, 0.0164272068175304, -0.008267546030807755)
q5 off
(-0.4590970494750445, 0.4593248581466239, 0.3517510096088717, 0.4143610444165538, 0.893369628255947, 3.124245611126277)
(2.0747336865245836, -0.8775399360111333, 0.8168694428775948, 0.0072423211177692435, 0.010580516424886529, -0.005744803846851343)
q5 okay
(-0.4627846037131499, 0.46943230157411603, 0.34076681275104, 2.5398106806113048, 0.9004312629178237, -2.737303947243086)
(2.082440715868149, -0.8887628760636037, 0.8139203680105475, 0.003393850514031889, 0.004772976480035287, -0.0031805375237060115)
q5 off
(-0.4664773393255812, 0.4795308958234648, 0.32976868942391674, 2.7196358750610536, 0.9074697087624131, 3.139535174536297)
(2.0900266991609633, -0.9000174169690957, 0.8108699712705107, -0.0004757252201534717, -0.000995028647004947, -0.0005746950760509344)
[INFO] [1499472696.445623, 1252.133000]: length of Joint Trajectory List: 37
[INFO] [1499473396.087126, 1892.594000]: Received 57 eef-poses from the plan
q5 off
(-0.466436398850603, 0.4794003098960569, 0.32984485984848005, 2.719549363442341, 0.9070930053301302, -1.919657759866803)
(2.089985231868383, -0.8998946933107822, 0.8108952895104541, -0.0009028014956610309, -0.0006369580742090549, -0.0005415047929017038)
q5 okay
(-0.5148855974575401, 0.49072599287245905, 0.2740147130228861, 2.478123774131757, 0.859486339869528, -2.700332303938634)
(2.0946839466774057, -1.022742142975975, 0.8464594913805087, -0.01869895452450051, 0.021906094162494665, -0.02900912158122216)
q5 off
(-0.5629261662923276, 0.502150628896204, 0.21859800914347277, 1.5043739825062663, 0.8126840075178694, -1.9090015452441194)
(2.0900887388099285, -1.1481286287923897, 0.8835489153750062, -0.03892054176159559, 0.0448261361066229, -0.06381396943774137)
q5 off
(-0.6105853253371323, 0.5136876450189719, 0.16377443899910366, 0.4324280610519966, 0.7670879618075425, 2.6659293326644757)
(2.0759582787633994, -1.2753254301073975, 0.9222001870155024, -0.06150703554108384, 0.06746625967700219, -0.10510352775757642)
q5 off
(-0.6579055346081369, 0.5253541416421271, 0.10973888503315049, 3.0753986322205598, 0.7232013521845482, -1.8305402633182362)
(2.0521019133533858, -1.4035650130545676, 0.9624559391425694, -0.08639920388853076, 0.08911853676482635, -0.1529384741850709)
q5 off
(-0.6696782389579282, 0.5283241618074065, 0.09542539131310251, 0.229532826812466, 0.7121583348169127, 2.3526545177172746)
(2.045228004966941, -1.4363521744536218, 0.9741231785292686, -0.09566029475459358, 0.09407907248795214, -0.1663365771257161)
q5 off
(-0.6814340318629665, 0.5313101437622, 0.08118909412032865, 1.436258097262049, 0.701341701481073, 2.6566416717004495)
(2.037694830858234, -1.4691457619124952, 0.9859163581057084, -0.10506873541286547, 0.09889944061129818, -0.18017788348740385)
q5 okay
(-0.7023594063889389, 0.5376350469249357, 0.03669516137769031, 2.33287159096795, 0.6773133017638195, -2.683102112337867)
(2.0338807341055176, -1.5413727912920963, 1.0358754213956811, -0.1775590375134425, 0.10203018831975554, -0.2159183022247637)
q5 off
(-0.7234766389594603, 0.5445315164150736, -0.007188613276917826, 1.5034364174382195, 0.6597297544995772, 1.9942149564556033)
(2.0258575902522047, -1.6131331019638961, 1.0874506152579426, -0.2505898645418222, 0.1033063068087944, -0.2546125020016946)
q5 off
(-0.7650664149525565, 0.5620987192499273, -0.11050575615226482, 2.8716758548021524, 0.6585613446665823, 2.0861792735848805)
(2.0071416471775105, -1.7593321087734617, 1.2234742255344753, -0.44998656372877416, 0.09072447609694877, -0.3488098261876938)
q5 off
(-0.8099577353265328, 0.5854041697133517, -0.21655906787602736, 3.0250953374423197, 0.7107812648625522, 1.1722502322529775)
(1.9674351485155548, -1.897730227701263, 1.367674595431435, -0.6469951238388341, 0.06402553751432517, -0.4543044641166654)
q5 okay
(-0.8708248077733184, 0.6257000870821726, -0.3673347253558254, 2.5735847220330488, 0.8436295181070819, 2.9556729531833983)
(1.8901731369098405, -2.0551945596806465, 1.5651677794066903, -0.8970879092114098, 0.0050047121120642525, -0.6004743586272688)
q5 off
(-0.9349291058890866, 0.6796421087029713, -0.5455152971192557, 2.5372348967100935, 1.0105967947712902, -2.8120423808385966)
(1.7813488571099465, -2.1879587815770463, 1.7707203600056094, -1.13028716111024, -0.08095462447613136, -0.7556058917127076)
q5 okay
(-0.9712653634592602, 0.7289726504433228, -0.7074125839744533, 0.31776866693402445, 1.1400334616126038, 2.8284747315089587)
(1.7089728220429452, -2.2416055721862738, 1.9345561488667642, -1.3170881377624308, -0.16558785048905642, -0.8607022531727346)
q5 off
(-1.0060488817564388, 0.7910258776711863, -0.8998415924353494, 0.14101806657986143, 1.2488337908832605, -2.820118673902961)
(1.6236212466768916, -2.277657613532369, 2.0980864081226183, -1.4882802430600037, -0.260720958737852, -0.9692718409219417)
q5 okay
(-1.0380079120985146, 0.8779475123005276, -1.1444793803720157, 2.606442033789663, 1.3147492816459134, 3.0981224345227862)
(1.5267167459746949, -2.2948135847152664, 2.2590384604736755, -1.6414706659912717, -0.3643956514761461, -1.0832099840126488)
d2_WC > invalid d2_WC
q5 okay
(-1.066319943367377, 1.0122473707465616, -1.4814561641348893, 2.7669943539630624, 1.303335026277387, -2.5203179224400136)
(1.4199372744325205, -2.2921158600782388, 2.41503770349907, -1.7735436516330405, -0.4741821109517258, -1.2058127419663682)
d2_WC > invalid d2_WC
q5 okay
(-1.067599704523117, 0.98160504439965, -1.4814561641348893, 2.677616475310195, 1.3687402839687577, -2.9897780739770434)
(1.3937448315416647, -2.256498691810005, 2.4999850994884785, -1.863953879314189, -0.5319388781860768, -1.2564071557128365)
d2_WC > invalid d2_WC
q5 off
(-1.0680316599343713, 0.9506731635637315, -1.4814561641348893, 3.045727557839529, 1.4215563066100474, 2.0113442949953084)
(1.3647377280010318, -2.2165713881250424, 2.5822259121963866, -1.9470763977453955, -0.5888165257279977, -1.3115069883981725)
d2_WC > invalid d2_WC
q5 off
(-1.0675308039059155, 0.9196057124720111, -1.4814561641348893, 1.7493601788345154, 1.4597539813696196, -1.727183271443678)
(1.3330004747573094, -2.172477553104464, 2.661424024344052, -2.022074033123167, -0.6442447022548518, -1.3720843244436236)
d2_WC > invalid d2_WC
q5 okay
(-1.06599551268271, 0.8884894576371936, -1.4814561641348893, 2.859407978340165, 1.481060769246184, 3.1208823918725774)
(1.298631243059572, -2.12438063056074, 2.73725370212442, -2.087995413770413, -0.6975457937390405, -1.439214762731142)
d2_WC > invalid d2_WC
q5 okay
(-1.0607622244620991, 0.884666171133689, -1.4814561641348893, 2.757402602370237, 1.4944370354973031, -2.847856694638709)
(1.3099054159605261, -2.109668495927367, 2.7448036822567055, -2.1067206960389804, -0.6998710418056079, -1.435294106950245)
d2_WC > invalid d2_WC
q5 off
(-1.0555375503949695, 0.8808002335397243, -1.4814561641348893, 2.6083656243360176, 1.507703578386078, -2.4756911156904904)
(1.3210211778305618, -2.094879143921087, 2.752324687971594, -2.1254368426705956, -0.7021719931709266, -1.4313709025053656)
d2_WC > invalid d2_WC
q5 okay
(-1.0194918780797566, 0.8665581326945911, -1.4814561641348893, 2.584544692937618, 1.6496968695241343, -2.886553917397645)
(1.4234715576169357, -2.0100851587874207, 2.767330898227901, -2.289841569615061, -0.7078902507203806, -1.3592231644127841)
d2_WC > invalid d2_WC
q5 off
(-0.981875536494835, 0.8476384622779884, -1.4814561641348893, 2.0938034237958925, 1.7759769335812392, -2.615961274446465)
(1.5205870543941322, -1.9202006664982911, 2.7822301562466922, -2.4544905743259915, -0.7136604366724149, -1.2869182711744884)
d2_WC > invalid d2_WC
q5 off
(-0.8635799129370327, 0.781243430455783, -1.4814561641348893, 1.478418010749926, 2.01750740980144, 2.5173918590782653)
(1.7646913082673767, -1.6574810440443737, 2.8114456287957914, -2.9120641419318254, -0.725288226975597, -1.0770623430672366)
d2_WC > invalid d2_WC
q5 off
(-0.7086042744431846, 0.7042075817795275, -1.4814561641348893, 1.6393827489362527, 2.0450613530951482, 2.3707825058578615)
(1.9645027326605664, -1.3642668934731275, 2.8398395590766357, 2.9119197654305853, -0.7355452957017165, -0.8660211703177549)
d2_WC > invalid d2_WC
q5 off
(-0.48499835850515916, 0.6375070369696028, -1.4814561641348893, 0.6729254414619483, 1.840006207196143, 2.6008402306942218)
(2.1331335099650506, -1.0110556086180833, 2.867679787915552, 2.39630631581897, -0.743585271369218, -0.6264218407941952)
d2_WC > invalid d2_WC
q5 off
(-0.23721365991957472, 0.616775632087571, -1.4814561641348893, -0.29426877410334884, 1.4871415085951316, 1.8403029694659419)
(2.2389568167589267, -0.6391250077292978, 2.894418562347295, 1.8792928019269977, -0.7491143885628374, -0.3858417894736706)
d2_WC > invalid d2_WC
q5 off
(-0.0940688267185993, 0.6291272505541206, -1.4814561641348893, -0.3652777280563655, 1.2138728198384676, 0.2378505223371091)
(2.27537324918641, -0.39685371560161775, 2.9082319489478863, 1.5471202051893898, -0.7493208438240317, -0.22922177391449422)
q5 off
(0.02251217139722666, 0.5349024268114949, -1.269167432291828, -0.8410255924749311, 0.8707105206168955, -0.39477152386561787)
(2.2854784153881567, -0.1538850014448079, 2.921557984127623, 1.2149131396742292, -0.7481236349942935, -0.07264532975273848)
q5 off
(0.08850599282976758, 0.43654955284323954, -1.0608017362144988, -0.9843241847778856, 0.6483381038433903, 2.3480251882339207)
(2.2785905218653517, 0.019550057804263755, 2.9286933331814864, 0.9758781771126823, -0.7446461177395158, 0.04140031756447675)
q5 off
(0.14242667411916005, 0.3838532074424897, -0.9462900367079959, -0.08952685264665503, 0.4751237750674777, 0.8209811907371052)
(2.2584328860261507, 0.1910976978205551, 2.93555596790974, 0.7371688069802003, -0.7402748539477407, 0.15513950860263687)
q5 off
(0.1612303797529504, 0.3649018344035642, -0.8896979772873651, -2.7195292901960735, 0.3495389339013348, -2.9846488174836567)
(2.2624280311294513, 0.28861429287962037, 2.917526960711024, 0.561386007188713, -0.7170402026603024, 0.23549051147203262)
q5 okay
(0.17934965342611123, 0.3562146333325807, -0.8604441724522943, -3.0133313778914514, 0.25400855499694086, -2.6646316356716575)
(2.2622019978096914, 0.38658411558698874, 2.8987024564491923, 0.38878971608897167, -0.6901186409768678, 0.312969294523007)
q5 okay
(0.22760302878488228, 0.3729570541967849, -0.8786811396006752, 2.5295133703423938, 0.30461542768309924, -1.8674596629838254)
(2.2595775257207014, 0.616242557771594, 2.835315754648845, -0.020249755779521355, -0.5949923024796154, 0.4851017803193828)
q5 okay
(0.3059329312069527, 0.43398158254021096, -0.998559406467371, 0.8554571394542375, 0.6235956034813608, -1.303820668405184)
(2.232911132434316, 0.8453327934869452, 2.7659001719481684, -0.40335672430634495, -0.47902095581146525, 0.630899654439367)
d2_WC > invalid d2_WC
q5 okay
(0.5604371189528364, 0.7242986921110546, -1.4814561641348893, -0.0107524193608133, 1.4696384969286995, -1.0245327687360135)
(2.1065008352295393, 1.318126137375085, 2.5835541160311326, -1.168442194779141, -0.1621853174451597, 0.8563099507159884)
d2_WC > invalid d2_WC
q5 okay
(0.8236372313186868, 0.8311147730916291, -1.4814561641348893, -0.7743297388455391, 1.9467448757890518, -1.0606365813517669)
(1.8767214783618669, 1.736031570213462, 2.385726071456045, -1.8944811858421549, 0.18479896064621945, 0.974315186636797)
d2_WC > invalid d2_WC
q5 okay
(1.0334530145121095, 0.9952480619282718, -1.4814561641348893, -1.7519876661985787, 2.0565761924512636, -1.4100817844722835)
(1.5497341651312875, 2.086082098679759, 2.1759352555630325, -2.7060335698089775, 0.5340170639625204, 0.9540611103561154)
q5 off
(1.1817794065341345, 0.6592200612376312, -0.6306215250877146, -2.4809460610093206, 1.6521010668050429, -1.9507797672041)
(1.1573553591556787, 2.3342165705450992, 1.988045828911718, 2.599549174727401, 0.7696428354248172, 0.7104947175734492)
q5 okay
(1.2668031218551443, 0.4749144381089559, -0.3268860462965743, -2.5716673819728104, 1.0586310807333768, -2.0020434444983457)
(0.9120868804874445, 2.4312587533916585, 1.891883736308508, 1.9516251011436798, 0.794267047433913, 0.46544174430691504)
q5 off
(1.3839759941811773, 0.39922093588624735, -0.2842140074887222, -2.9314721808235977, 0.6689192265791688, -3.038089712524815)
(0.6551098540599284, 2.4962082039841267, 1.8092493055742267, 1.3380319503409783, 0.7160215138700853, 0.24171925718094425)
q5 off
(1.4676824297759101, 0.4259126660510016, -0.3457738390691882, -2.9671715291301823, 0.865012853023811, 2.7228678305970697)
(0.48828985493356447, 2.5213757968635684, 1.7614187647014488, 0.9933096780938067, 0.6176411511388133, 0.13537004399224722)
q5 okay
(1.5496350134926329, 0.48019778924408124, -0.4160668667826419, -1.2166020502671548, 1.1681694084394858, 2.2794196219899905)
(0.3174360164660758, 2.5343644741712805, 1.7186598632478696, 0.692024931863776, 0.49485775081213174, 0.06636413599934603)
q5 off
(1.577584271188371, 0.5010547316921966, -0.43680734686009876, -2.928599443122243, 1.270303786958134, 2.1570733298049287)
(0.25644877834136126, 2.5360594969776056, 1.704147014545909, 0.5945497033829111, 0.4456215608118541, 0.04878584993037284)
q5 okay
(1.6050100115647228, 0.521003465266146, -0.4536969303606195, -1.6899617566732357, 1.364579882264701, 2.1080097048189654)
(0.19496967392913814, 2.5362203907979235, 1.6901532970551119, 0.5016723786241276, 0.3946060242809973, 0.03486855351914158)
q5 okay
(1.6319680919146715, 0.5394160958865944, -0.46627413397079737, -1.2680231031040998, 1.4500942794212104, 2.0445078859794332)
(0.13301533199149773, 2.53482922493515, 1.6766305150178995, 0.4130648019006757, 0.3420911697168112, 0.024226497834348885)
q5 okay
(1.6585322224430283, 0.5558533759923784, -0.4743116977525172, -1.306420597000705, 1.5264657210643764, 1.989058172461551)
(0.07060626897936312, 2.531863998371478, 1.66352796874906, 0.32842555758529035, 0.2883241525416246, 0.016491702043745667)
q5 off
(1.6773704563664924, 0.5675840288019637, -0.4770778826875039, -3.125797227614116, 1.5794964079390164, 2.3470530863583408)
(0.02595410245626198, 2.5289001461142666, 1.652820047087306, 0.2697096762168441, 0.24287555307078498, 0.010447955798563065)
q5 okay
(1.6960592208421283, 0.5778770625897182, -0.47711330389333634, -1.3801181744399997, 1.627018840714486, 1.9756950882086999)
(-0.01902404068329952, 2.5251073108452475, 1.6423240632185365, 0.21327968694723562, 0.1968722988287309, 0.005942316518538104)
q5 = pi-q5
(1.7146334903988065, 0.5867194340825638, -0.4745450132297941, -0.06144717337243086, 1.6692141801692717, 1.8384115785124358)
(-0.06431726092767118, 2.5204693537857406, 1.6320097943241676, 0.15901861551483298, 0.15040723798646594, 0.0028082894208202778)
q5 off
(1.7331264655579428, 0.5941305334916623, -0.4695332436700545, -1.4675542013298635, 1.706294239521823, 1.8000762756777626)
(-0.10991218401692857, 2.514968859397263, 1.6218466309654653, 0.10682577816718744, 0.10356525754431652, 0.0008897311717504348)
q5 off
(1.7431571921473399, 0.5975397731857894, -0.4657768803829833, -1.492749637445273, 1.72423867144956, 1.7801224229808774)
(-0.1348130854182802, 2.511616835036762, 1.616357298374177, 0.07930953643334232, 0.07793434261357782, 0.00046121621045392043)
q5 off
(1.7531778388012562, 0.6005486964863535, -0.46138594289682805, -1.518320350956608, 1.7407796808000482, 1.7607386911290077)
(-0.15979538464311732, 2.5080013267116543, 1.6108984824836152, 0.05236044160757044, 0.05222797649717825, 0.00032670605932586997)
q5 off
(1.7631928461312105, 0.6031687141707528, -0.45639348937110075, -1.544158700084601, 1.7559585335029022, 1.6697245247540995)
(-0.18485593537542166, 2.5041191788480686, 1.6054652567632208, 0.025967997751084073, 0.02645730822030054, 0.000464444173603811)
q5 off
(1.7732064141646662, 0.6054125377138667, -0.45083342905187157, -3.1109842280777755, 1.7698178928015538, -3.141469129247136)
(-0.2099913456238965, 2.4999671801967827, 1.6000527060514356, 0.00012298419486813525, 0.0006330890877831505, 0.0008532257478854158)
[INFO] [1499473475.863460, 1951.256000]: length of Joint Trajectory List: 57

